In [16]:
import numpy as np
import gvar as gv
import sys

#import warnings  
#warnings.simplefilter(action="default")

sys.path.append('../')
#import xpt
from h5glance import H5Glance 
import os
import lsqfit

In [17]:
os.chdir('../../hyperon-xpt-master/xpt')

In [18]:
os.getcwd()

'/Users/grantdb/hyperon-xpt-master/xpt'

In [4]:
#import importlib
#importlib.reload(xpt.io)
import i_o
#import non_analytic_functions as naf 
#reload(fm)
#reload(fit)

In [5]:
lam_chi = i_o.InputOutput.get_data_phys_point('lam_chi')
print(lam_chi)

{'a/w': 0(0), 'a': 0(0), 'alpha_s': 0(0), 'L': inf(0), 'hbarc': 197.327(0), 'lam_chi': 1157.0(7.2), 'm_pi': 134.80(30), 'm_k': 494.20(30), 'm_xi': 1318.28(11), 'm_xi_st': 1533.40(34)}


In [19]:
file = '../data/hyperon_data.h5'
H5Glance(file)

In [7]:
file2 = '../data/input_data.h5'
H5Glance(file2)

In [8]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_imp')

for j,ens in enumerate(ensembles):
    print(ens, data['m_pi'][j])

/Users/grantdb/hyperon-xpt-master/xpt/i_o.py:89: UserWarning: sample sizes differ for different entries: 5000 5001
  gv_data[ens] = gv.dataset.avg_data(gv_data[ens], bstrap=True)
a09m135 134.4(1.1)
a09m220 221.3(1.8)
a09m310 318.1(2.6)
a09m350 356.8(2.9)
a09m400 409.5(3.3)
a12m130 132.9(1.0)
a12m220L 219.2(1.6)
a12m310 308.6(2.3)
a12m350 349.9(2.6)
a12m400 398.2(2.9)
a15m135XL 134.96(90)
a15m220 216.8(1.5)
a15m310 309.5(2.1)
a15m350 347.1(2.3)
a15m400 397.0(2.7)


In [9]:
data.keys()

dict_keys(['m_delta', 'm_lam', 'm_sigma', 'm_sigma_st', 'm_xi', 'm_xi_st', 'm_pi', 'm_k', 'lam_chi', 'a/w'])

In [10]:
data['m_xi']

array([1282(12), 1318(11), 1370(11), 1388(11), 1411(12), 1271.5(9.6),
       1283(10), 1318.6(9.9), 1336(10), 1358(10), 1258.0(8.6),
       1257.6(8.5), 1294.3(8.7), 1302.1(9.7), 1327.5(9.8)], dtype=object)

In [11]:
#data,ensembles = input_output.get_data(scheme='w0_imp')
physical_point = input_output.get_data_phys_point()
print(physical_point)

{'a/w': 0(0), 'a': 0(0), 'alpha_s': 0(0), 'L': inf(0), 'hbarc': 197.327(0), 'lam_chi': 1157.0(7.2), 'm_pi': 134.80(30), 'm_k': 494.20(30), 'm_xi': 1318.28(11), 'm_xi_st': 1533.40(34)}


In [12]:
prior = {}
# not-even leading order 
prior['m_{xi,0}'] = gv.gvar(1000, 1000) # MeV
prior['m_{xi_st,0}'] = gv.gvar(1100, 1000) # MeV
# nlo
prior['s_{xi}'] = gv.gvar(0, 5)
prior['s_{xi,bar}'] = gv.gvar(0, 5)
# n2lo
prior['g_{xi,xi}'] = gv.gvar(0.25, 0.25)
prior['g_{xi_st,xi}'] = gv.gvar(0.25, 0.25)
prior['g_{xi_st,xi_st}'] = gv.gvar(0, 3)
# n3lo
prior['b_{xi,4}'] = gv.gvar(0, 5)
prior['b_{xi_st,4}'] = gv.gvar(0, 5)
prior['a_{xi,4}'] = gv.gvar(0, 5)
prior['a_{xi_st,4}'] = gv.gvar(0, 5)
# latt/strange nlo
prior['d_{xi,a}'] = gv.gvar(0, 1)
prior['d_{xi_st,a}'] = gv.gvar(0, 1) 
prior['d_{xi,s}'] = gv.gvar(0, 3)
prior['d_{xi_st,s}'] = gv.gvar(0, 3) 
# latt n2lo
prior['d_{xi,aa}'] = gv.gvar(0, 3)
prior['d_{xi,al}'] = gv.gvar(0, 6)
prior['d_{xi,as}'] = gv.gvar(0, 3)
prior['d_{xi,ls}'] = gv.gvar(0, 3)
prior['d_{xi,ss}'] = gv.gvar(0, 3)

prior['d_{xi_st,aa}'] = gv.gvar(0, 3)
prior['d_{xi_st,al}'] = gv.gvar(0, 3) 
prior['d_{xi_st,as}'] = gv.gvar(0, 3)
prior['d_{xi_st,ls}'] = gv.gvar(0, 3) 
prior['d_{xi_st,ss}'] = gv.gvar(0, 3)

In [13]:
def _make_prior(data, prior):
    new_prior = {}
    for key in prior:
        new_prior[key] = prior[key]
    for key in ['m_pi', 'm_k', 'lam_chi', 'a/w']:
        new_prior[key] = data[key]
    return new_prior

In [14]:
_make_prior(data,prior)


{'m_{xi,0}': 1000(1000),
 'm_{xi_st,0}': 1100(1000),
 's_{xi}': 0.0(5.0),
 's_{xi,bar}': 0.0(5.0),
 'g_{xi,xi}': 0.25(25),
 'g_{xi_st,xi}': 0.25(25),
 'g_{xi_st,xi_st}': 0.0(3.0),
 'b_{xi,4}': 0.0(5.0),
 'b_{xi_st,4}': 0.0(5.0),
 'a_{xi,4}': 0.0(5.0),
 'a_{xi_st,4}': 0.0(5.0),
 'd_{xi,a}': 0.0(1.0),
 'd_{xi_st,a}': 0.0(1.0),
 'd_{xi,s}': 0.0(3.0),
 'd_{xi_st,s}': 0.0(3.0),
 'd_{xi,aa}': 0.0(3.0),
 'd_{xi,al}': 0.0(6.0),
 'd_{xi,as}': 0.0(3.0),
 'd_{xi,ls}': 0.0(3.0),
 'd_{xi,ss}': 0.0(3.0),
 'd_{xi_st,aa}': 0.0(3.0),
 'd_{xi_st,al}': 0.0(3.0),
 'd_{xi_st,as}': 0.0(3.0),
 'd_{xi_st,ls}': 0.0(3.0),
 'd_{xi_st,ss}': 0.0(3.0),
 'm_pi': array([134.4(1.1), 221.3(1.8), 318.1(2.6), 356.8(2.9), 409.5(3.3),
        132.9(1.0), 219.2(1.6), 308.6(2.3), 349.9(2.6), 398.2(2.9),
        134.96(90), 216.8(1.5), 309.5(2.1), 347.1(2.3), 397.0(2.7)],
       dtype=object),
 'm_k': array([493.9(4.0), 517.0(4.2), 544.9(4.4), 558.2(4.5), 576.9(4.6),
        494.1(3.6), 507.3(3.7), 530.1(3.9), 544.7(4.0), 561

In [20]:
model_info = {}
model_info['particles'] = ['xi','xi_st']
model_info['name'] = 'test'
model_info['order_chiral']  = 'n2lo'
model_info['order_disc'] = 'n2lo'
model_info['order_strange'] = 'n2lo'
#model_info['eps2a_defn'] = 't0_original'
model_info['xpt'] = True


NameError: name 'particles' is not defined

In [14]:
print(model_info)



In [15]:

data,ensembles = input_output.get_data(scheme='w0_imp')
#model_info = data_loader.get_model_info_from_name('Fpi_n3lo_fv_w0impr')
prior = _make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

NameError: name 'dl' is not defined

In [ ]:
fit_manager = fm.fit_manager(
    phys_point_data=phys_point_data, 
    fit_data=data, 
    model_info=model_info,
    prior=prior)

In [ ]:
print(fit_manager)